In [9]:
import pandas as pd
import unidecode

file = '../data/01_raw/2674.xls'

# Dicionário com o range de linhas de cada continente no arquivo
continent_range = {
    'África': '9:11',
    'América Central e Caribe': '21:11',
    'América do Norte': '33:3',
    'América do Sul': '37:12',
    'América do Sul': '37:12',
    'Ásia': '50:19',
    'Europa': '70:33',
    'Oceania': '104:3',
    'Países não especificados': '107:1',
}


def sanitize_file_name(file_name):
    """
    Converte o nome do arquivo para um formato padronizado
    """
    file_name = unidecode.unidecode(file_name)
    return file_name.lower().replace(' ', '_')


def get_continent_data(continent, year='2019'):
    """
    Retorna um DataFrame normalizado com os dados de um continente específico
    """
    start, end = continent_range[continent].split(':')
    df = pd.read_excel(file, year, skiprows=int(
        start), nrows=int(end),  usecols='A:D')
    df.columns = [continent, 'Total', 'Aérea', 'Marítima']

    # Troca '-' por 0 nas colunas Toal, Aérea e Marítima
    df['Total'] = df['Total'].replace('-', 0)
    df['Aérea'] = df['Aérea'].replace('-', 0)
    df['Marítima'] = df['Marítima'].replace('-', 0)

    # Parse as colunas para inteiros
    df['Total'] = df['Total'].astype(int)
    df['Aérea'] = df['Aérea'].astype(int)
    df['Marítima'] = df['Marítima'].astype(int)

    # Adiciona a coluna 'Ano' com o valor do ano
    df['Ano'] = year

    return df

In [10]:
# Salva os dados de cada continente em um arquivo CSV
# for continent in continent_range:
#     df = get_continent_data(continent)
#     df.to_csv(
#         f'../data/02_processed/{sanitize_file_name(continent)}.csv', index=False)

In [11]:
# Salva os dados de todos os continentes em um arquivo CSV
# Ignorando a primeira linha que contém o total de todos os continentes
# Onde teremos País,Total,Aérea,Marítima,Continente
columns = ['País', 'Continente', 'Aérea', 'Marítima', 'Total', 'Ano']
df = pd.DataFrame(columns=columns)


# Years
years = ['2016', '2017', '2018', '2019']

for year in years:
    for continent in continent_range:
        df_continent = get_continent_data(continent, year=year)
        # Rename first column to 'País'
        df_continent.columns = ['País', 'Total', 'Aérea', 'Marítima', 'Ano']

        # Adiciona a coluna 'Continente' com o nome do continente
        df_continent['Continente'] = continent

        # Normalize 'País' column
        # If the country has a "," keep only the last part
        df_continent['País'] = df_continent['País'].apply(
            lambda x: x.split(',')[-1].strip() if ',' in x else x)

        # Se o row do País conter "Outros países" substitui por "Outros"
        df_continent['País'] = df_continent['País'].apply(
            lambda x: 'Outros' if 'Outros países' in x else x)

        # Adiciona a coluna 'Ano' com o valor do ano
        df_continent['Ano'] = year

        # Reordena as colunas
        df_continent = df_continent[columns]
        df = pd.concat([df, df_continent])

df.to_csv(f'../data/02_processed/total_continentes.csv', index=False)
df

C:\Users\rafae\AppData\Local\Temp\ipykernel_11984\564372660.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Marítima'] = df['Marítima'].replace('-', 0)
C:\Users\rafae\AppData\Local\Temp\ipykernel_11984\564372660.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Marítima'] = df['Marítima'].replace('-', 0)
C:\Users\rafae\AppData\Local\Temp\ipykernel_11984\564372660.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call

,País,Continente,Aérea,Marítima,Total,Ano
0,África do Sul,África,1660,15,1675,2016
1,Angola,África,6497,0,6497,2016
2,Cabo Verde,África,137,0,137,2016
3,Egito,África,1069,2,1071,2016
4,Gana,África,120,0,120,2016
...,...,...,...,...,...,...
32,Outros,Europa,1649,39,1688,2019
0,Austrália,Oceania,12929,219,13148,2019
1,Nova Zelândia,Oceania,1734,30,1764,2019
2,Outros,Oceania,31,4,35,2019
